In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 


In [15]:
riskDebtAccounts_files = [x for x in os.listdir() if 'risk' in x and '.xl' in x and '~' not in x]
riskDebtAccounts_files

['riskDebtAccounts_(16).xlsx']

In [87]:
data_riskDebtAccounts = pd.read_excel(riskDebtAccounts_files[0], header=5)
data_riskDebtAccounts.head()

,active,action,type,nameIFRS,codeIFRS,sourceSystem,accountCode,accountPlanType,reportType,bscCode,bsxCode,overdueFlag,ocpFlag,resetOverdueFlag
0,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в евро,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX303,NaN,Y,NaN
1,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в браз...,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX304,NaN,Y,NaN
2,Y,Include,Trade payables,Задолженность поставщикам и подрядчикам в проч...,B601420,ETL document loader,NaN,NaN,AP,BSC300,BSX309,NaN,Y,NaN
3,Y,Include,Trade payables,Кредиторская задолженность за основные средств...,B601300,ETL document loader,NaN,NaN,AP,BSC300,BSX371,NaN,Y,NaN
4,Y,Include,Trade payables,НДС по кредиторской задолженности за основные ...,B769926,ETL document loader,NaN,NaN,AP,BSC300,BSX372,NaN,Y,NaN


In [88]:
data_riskDebtAccounts_work = data_riskDebtAccounts[:]
# xxmdg."riskDebtAccounts"."ocpFlag" = 'Y'
# xxmdg."riskDebtAccounts"."active" = 'Y'
# xxmdg."riskDebtAccounts"."action" = 'Include'
data_riskDebtAccounts_work = data_riskDebtAccounts_work[(data_riskDebtAccounts_work.ocpFlag=='Y')&(data_riskDebtAccounts_work.active=='Y')&(data_riskDebtAccounts_work.action=='Include')]
data_riskDebtAccounts_work = data_riskDebtAccounts_work[['type','sourceSystem','accountCode','reportType','bscCode']]
# pivots
pivot_1 = data_riskDebtAccounts_work.groupby(['bscCode','accountCode'],dropna=False,as_index=False)[['reportType','sourceSystem','type']].agg(lambda x: str(set(x)))
pivot_2 = pivot_1.groupby(['sourceSystem', 'type','reportType'],dropna=False,as_index=False)[['bscCode','accountCode']].agg(lambda x: str(set(x)))
pivot_2 = pivot_2.replace({'{':'', '}':'', '\"':'', 'nan':np.nan}, regex=True)
# filter to 'AR', 'AP' because of SQL error
pivot_2 = pivot_2[(pivot_2.reportType=='\'AR\'')|(pivot_2.reportType=='\'AP\'')]
pivot_2.head(5)

,sourceSystem,type,reportType,bscCode,accountCode
0,'OEBS12_EU','Trade receivables','AR','BSC130','015210'
1,"'OEBS12_SA', 'OEBS12_EU', 'ETL document loader...",'Other payables','AP',"'BSC310', 'BSC315'",NaN
2,"'OEBS12_SA', 'OEBS12_EU', 'ETL document loader...",'Provisions','AP','BSC330',NaN
3,"'OEBS12_SA', 'OEBS12_EU', 'ETL document loader...",'Trade payables','AP',"'BSC300', 'BSC305'",NaN
4,"'OEBS12_SA', 'OEBS12_EU', 'ETL document loader'",'Other receivables','AR',"'BSC140', 'BSC145'",NaN


In [89]:
pivot_2_SQL = pivot_2[:]
# to sql statements
pivot_2_SQL.sourceSystem = pivot_2_SQL.sourceSystem.apply(lambda x: f'WHEN "sourceSystem" in ({x})')
pivot_2_SQL.reportType = pivot_2_SQL.reportType.apply(lambda x: f'and "reportType" in ({x})')
pivot_2_SQL.loc[~pivot_2_SQL.bscCode.isna(), 'bscCode'] = pivot_2_SQL.loc[~pivot_2_SQL.bscCode.isna(), 'bscCode'].apply(lambda x: f'and "bscCode" in ({x})')
pivot_2_SQL.loc[~pivot_2_SQL.accountCode.isna(), 'accountCode'] = pivot_2_SQL.loc[~pivot_2_SQL.accountCode.isna(), 'accountCode'].apply(lambda x: f'and "glAccount" in ({x})')
pivot_2_SQL.type = pivot_2_SQL.type.apply(lambda x: f'THEN {x}')
# to SQL
pivot_2_SQL = pivot_2_SQL[['sourceSystem','reportType','bscCode','accountCode','type']]
case_statement = pivot_2_SQL.to_string(header=False, index=False, na_rep='')
### QUERY
query = f"""CASE
            {case_statement}
            END AS "accountPurpose"
            from RISKACCESS."unifiedARReport" 
            where "rowType" = 'M' 
"""
# to text
with open(f'{str(date.today())}_SQL_query.txt', 'w') as file:
    file.write(query)